In [1]:
from google.cloud import storage
import pandas as pd
import io

1. Create Client Object

In [2]:
try:
     # creating the client object
    SERVICE_ACCOUNT_JSON=r'D:\project\aerial-reef-332517-07a94b2ffe42.json'
    storage_client=storage.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)
except Exception as e:
    print("ERROR:{}".formate(e))

In [3]:
print(storage_client)

2. Getting all the files from Gcp bucket

In [4]:
bucket=storage_client.get_bucket('gcp_proj_temp')

In [5]:
temp=[]
for filename in list(bucket.list_blobs(prefix='')):
    print(filename)
    temp.append(filename.name)
print(temp)

<Blob: gcp_proj_temp, Retail_Data_Transactions.csv, 1637517878202520>
['Retail_Data_Transactions.csv']


In [8]:
#Uploading csv file to GCS BUCKET
import os

filename='user.json'
UPLOAD_FILE_NAME=os.path.join(filename)

#bucket
bucket=storage_client.get_bucket('gcp_proj_temp')
blob=bucket.blob(filename)
blob.upload_from_filename(UPLOAD_FILE_NAME)

print("Successfully uploaded the {} file to GCS bucket".format(filename))

Successfully uploaded the user.json file to GCS bucket


4.Downloading the FIle from GCS Bucket to local machine

In [22]:
def download_file_from_gcs(Bucket_name,source_blob_name,destination_file_name,storage_client):
    try:
        bucket=storage_client.bucket(Bucket_name)
        blob=bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)
        print(
            "Downloaded storage object {} from bucket {} to local file {}.".format(
                source_blob_name, Bucket_name, destination_file_name
            ))
    except Exception as e:
        print("ERROR : {}".format(e))

In [23]:
Bucket_name='gcp_proj_temp'
source_blob_name='Retail_Data_Transactions.csv'
destination_file_name='R_D_T.csv'
download_file_from_gcs(Bucket_name,source_blob_name,destination_file_name,storage_client)

Downloaded storage object Retail_Data_Transactions.csv from bucket gcp_proj_temp to local file R_D_T.csv.


5. Loading BigQuery table from csv file in GCS.

5.1 Creating data set

In [53]:
def create_dataset(dataset_name,location,description):
    from google.cloud import bigquery
    SERVICE_ACCOUNT_JSON=r'D:\project\aerial-reef-332517-07a94b2ffe42.json'
    storage_client=bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)
    dataset = bigquery.Dataset(dataset_name)
    dataset.location =location
    dataset = storage_client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(storage_client.project, dataset.dataset_id))

In [55]:
dataset_name="aerial-reef-332517.Retail_Data_temp"
loaction="US"
description="Dataset from python"
print(storage_client)
create_dataset(dataset_name,loaction,description)

Created dataset aerial-reef-332517.Retail_Data_temp


5.2 Creating and loading data into table from GCS bucket.

In [57]:
def load_table(table_id):
    from google.cloud import bigquery
    SERVICE_ACCOUNT_JSON=r'D:\project\aerial-reef-332517-07a94b2ffe42.json'
    storage_client=bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("Customer_id", "STRING"),
            bigquery.SchemaField("Transaction_date", "STRING"),
            bigquery.SchemaField("Transaction_amount", "INT64"),
        ],skip_leading_rows=1, source_format=bigquery.SourceFormat.CSV,)
    
    uri = "gs://gcp_proj_temp/Retail_Data_Transactions.csv"
    load_job = storage_client.load_table_from_uri(uri, table_id, job_config=job_config)
    load_job.result()
    destination_table = storage_client.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

In [58]:
# table_id = "your-project.your_dataset.your_table_name"
table_id="aerial-reef-332517.Retail_Data_temp.Retail_Data_Transactions"
load_table(table_id)

Loaded 125000 rows.


6.Parsing complex JSON files

In [42]:
import pandas as pd
import json

with open('./source_data.json') as access_json:
    data= json.load(access_json)

result=data['results']
result

[{'_class': 'question',
  'course': {'_class': 'course',
   'title': 'Angular 7 (formerly Angular 2) - The Complete Guide',
   'url': '/the-complete-guide-to-angular-2/'},
  'replies': [{'_class': 'answer',
    'user': {'_class': 'user', 'display_name': 'Maximilian Schwarzmueller'}}]},
 {'_class': 'question',
  'course': {'_class': 'course',
   'title': 'Angular 7 (formerly Angular 2) - The Complete Guide',
   'url': '/the-complete-guide-to-angular-2/'},
  'replies': [{'_class': 'answer',
    'user': {'_class': 'user', 'display_name': 'Maximilian Schwarzmueller'}},
   {'_class': 'answer',
    'user': {'_class': 'user', 'display_name': 'Daniel Weat'}}]},
 {'_class': 'question',
  'course': {'_class': 'course',
   'title': 'Angular 7 (formerly Angular 2) - The Complete Guide',
   'url': '/the-complete-guide-to-angular-2/'},
  'replies': [{'_class': 'answer',
    'user': {'_class': 'user', 'display_name': 'Maximilian Schwarzmueller'}},
   {'_class': 'answer',
    'user': {'_class': 'user'

In [38]:
Class=[]
course_name=[]
url=[]
replies=[]
StudentName=[]

In [39]:
for i in result:
    Class.append(i['_class'])
    course_details=i['course']
    course_name.append(course_details['title'])
    url.append(course_details['url'])
    replies_details=i['replies']
    user=[]
    reply=[]
    for j in replies_details:
        reply.append(j['_class'])
        user.append(j['user']['display_name'])
    StudentName.append(user)
    replies.append(reply)

print(len(Class))
print(len(course_name))
print(len(url))
print(len(replies))
print(len(StudentName))

5
5
5
5
5


In [40]:
df=pd.DataFrame({
    'Class':Class,
    'Course Name':course_name,
    'Course url':url,
    'Student Reply':replies,
    'studen Name':StudentName
})

In [41]:
df

,Class,Course Name,Course url,Student Reply,studen Name
0,question,Angular 7 (formerly Angular 2) - The Complete ...,/the-complete-guide-to-angular-2/,[answer],[Maximilian Schwarzmueller]
1,question,Angular 7 (formerly Angular 2) - The Complete ...,/the-complete-guide-to-angular-2/,"[answer, answer]","[Maximilian Schwarzmueller, Daniel Weat]"
2,question,Angular 7 (formerly Angular 2) - The Complete ...,/the-complete-guide-to-angular-2/,"[answer, answer, answer, answer]","[Maximilian Schwarzmueller, Mike Jauranol, Mik..."
3,question,Angular 7 (formerly Angular 2) - The Complete ...,/the-complete-guide-to-angular-2/,[answer],[Maximilian Schwarzmueller]
4,question,Angular 7 (formerly Angular 2) - The Complete ...,/the-complete-guide-to-angular-2/,"[answer, answer, answer]","[Maximilian Schwarzmueller, Bernd Raucher, Max..."
